In [2]:
from tensorflow.keras import datasets
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dropout, Conv1D, GlobalMaxPooling1D, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

In [3]:
vocab_size=10000
(x_train,y_train),(x_test, y_test) = datasets.imdb.load_data(num_words = vocab_size)

<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:155: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:156: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 

In [4]:
max_len = 500
x_train = pad_sequences(x_train, maxlen = max_len, padding='pre')
x_test = pad_sequences(x_test, maxlen = max_len, padding='pre')

In [5]:
x_train

array([[   0,    0,    0, ...,   19,  178,   32],
       [   0,    0,    0, ...,   16,  145,   95],
       [   0,    0,    0, ...,    7,  129,  113],
       ...,
       [   0,    0,    0, ...,    4, 3586,    2],
       [   0,    0,    0, ...,   12,    9,   23],
       [   0,    0,    0, ...,  204,  131,    9]], dtype=int32)

In [6]:
x_test

array([[   0,    0,    0, ...,   14,    6,  717],
       [   0,    0,    0, ...,  125,    4, 3077],
       [  33,    6,   58, ...,    9,   57,  975],
       ...,
       [   0,    0,    0, ...,   21,  846, 5518],
       [   0,    0,    0, ..., 2302,    7,  470],
       [   0,    0,    0, ...,   34, 2005, 2643]], dtype=int32)

In [7]:
model=Sequential()
model.add(Embedding(vocab_size, 256))
model.add(Dropout(0.2))
model.add(Conv1D(256,3, padding='valid',activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

In [8]:
es = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, patience = 3)
mc = ModelCheckpoint('CNN_premodel.h5', monitor = 'val_acc', mode = 'max', verbose = 1, save_best_only = True)

In [9]:
model.compile(optimizer='adam', loss = 'binary_crossentropy', metrics = ['acc'])
history = model.fit(x_train, y_train, epochs = 10, validation_data = (x_test, y_test), callbacks=[es, mc])

Epoch 1/10
782/782 [==============================] - 42s 51ms/step - loss: 0.3730 - acc: 0.8236 - val_loss: 0.2583 - val_acc: 0.8922

Epoch 00001: val_acc improved from -inf to 0.89224, saving model to CNN_premodel.h5
Epoch 2/10
782/782 [==============================] - 38s 49ms/step - loss: 0.1687 - acc: 0.9338 - val_loss: 0.2618 - val_acc: 0.8942

Epoch 00002: val_acc improved from 0.89224 to 0.89416, saving model to CNN_premodel.h5
Epoch 3/10
782/782 [==============================] - 38s 49ms/step - loss: 0.0661 - acc: 0.9784 - val_loss: 0.3155 - val_acc: 0.8900

Epoch 00003: val_acc did not improve from 0.89416
Epoch 4/10
782/782 [==============================] - 38s 49ms/step - loss: 0.0291 - acc: 0.9902 - val_loss: 0.4782 - val_acc: 0.8763

Epoch 00004: val_acc did not improve from 0.89416
Epoch 00004: early stopping


In [10]:
loaded_model = load_model('CNN_premodel.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(x_test, y_test)[1]))

782/782 [==============================] - 6s 8ms/step - loss: 0.2618 - acc: 0.8942

 테스트 정확도: 0.8942
